# Practical 03 - RRT Algorithm

Note: CoLab is not supported for this week. For the online version, please visit https://mybinder.org/v2/gh/tianleimin/ECE4078_Practical_2021/main and proceed as if running locally. **Remember to download a copy of the notebook when you are done if running online. Otherwise, you will lose the notebook once you close the tab.**

In [1]:
%matplotlib notebook

import math
import random
import os
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import time

# Import dependencies and set random seed
seed_value = int(time.time())#5
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)


from helper_james import *

global ani

In [4]:
# build obstacle list from map file

gt_dict = {"apple_0": {"y": -0.8229359984397888, "x": 0.5527273416519165}, "lemon_0": {"y": 0.7714277505874634, "x": 0.8121515512466431}, "person_0": {"y": 0.8801784515380859, "x": -0.9843060374259949}, "apple_1": {"y": -1.0844264030456543, "x": 1.435594916343689}, "lemon_1": {"y": -1.3594268560409546, "x": -0.7248011231422424}, "person_1": {"y": 1.1036710739135742, "x": 0.792996346950531}, "apple_2": {"y": -1.1520918607711792, "x": 1.027390956878662}, "lemon_2": {"y": -0.8656513094902039, "x": 0.13186204433441162}, "person_2": {"y": 1.1036710739135742, "x": 0.792996346950531}}





def generate_object_circle_lists(object_array, aruco_markers):
    apple_gt, lemon_gt, person_gt, marker_gt = [], [], [], []

    map1 = [apple_gt, lemon_gt, person_gt]
    for i in range(len(object_array)):
        for j in range(len(object_array[i])):
            map1[i].append([object_array[i][0], object_array[i][1], j])
    
    r_true_apple = 0.075
    r_true_lemon = 0.06
    r_true_person = 0.19
    r_true_marker = 0.1

    scale = 0.08

    r_true_apple += scale
    r_true_lemon += scale
    r_true_person += scale
    r_true_marker += scale

    all_obstacles = []
    for entry in apple_gt:
        all_obstacles.append(CircleT(entry[0], entry[1], r_true_apple, 0))

    for entry in lemon_gt:
        all_obstacles.append(CircleT(entry[0], entry[1], r_true_lemon, 1))

    for entry in person_gt:
        all_obstacles.append(CircleT(entry[0], entry[1], r_true_person, 2))

    for entry in marker_gt:
        all_obstacles.append(CircleT(entry[0], entry[1], r_true_marker, 3))

    return all_obstacles, apple_gt, lemon_gt, person_gt, marker_gt


def objects_not_done(apple_gt, lemon_gt, person_gt):
    goal_thresh = 0.6
    
    for person_idx, person in enumerate(person_gt):
        for apple_idx, apple in enumerate(apple_gt):
            if apple[0] in apples_done:
                continue
            elif np.linalg.norm(apple[0:2] - person[0:2]) < goal_thresh:
                #if this is the case, an apple is within range of a person, including some consideration of error
                apples_done.append(apple[0])
                people_done.append(person[0])
                pairs.append((person, apple))
                
    people_not_done = [n for n in person_gt if n[0] not in people_done]
    apples_not_done = [n for n in apple_gt if n[0] not in apples_done]

    lemons_not_done = []
    for lemon in lemon_gt:
        for person in person_gt:
            if np.linalg.norm(person[0:2] - lemon[0:2]) < goal_thresh:
                lemons_not_done.append(lemon)
                break
                
    return apples_not_done, lemons_not_done, people_not_done



apple_gt, lemon_gt, person_gt, marker_gt = [], [], [], []


for key in gt_dict:
    if key.startswith('apple'):
        apple_id = key.split('_')[1]
        apple_gt.append(np.array(list(gt_dict[key].values())[::-1] + [apple_id], dtype=float))
    elif key.startswith('lemon'):
        lemon_id = key.split('_')[1]
        lemon_gt.append(np.array(list(gt_dict[key].values())[::-1] + [lemon_id], dtype=float))
    elif key.startswith('person'):
        person_id = key.split('_')[1]
        person_gt.append(np.array(list(gt_dict[key].values())[::-1] + [person_id], dtype=float))
    elif key.startswith('aruco'):
        aruco_id = key.split('_')[0].split('aruco')[1]
        marker_gt.append(np.array(list(gt_dict[key].values())[::-1] + [aruco_id], dtype=float))

print(lemon_gt)


## obstacle list complete, we now want to generate pairings of people and apples, and mark which lemons need to be moved

required_distance = 0.5
goal_error = 0.05

## dont move apples that are already in place, find these pairs

#people_done = []
#apples_done = []
#pairs = []


#for person_idx, person in enumerate(person_gt):
#    for apple_idx, apple in enumerate(apple_gt):
#        if apple[0] in apples_done:
#            continue
#        elif np.hypot(apple[0] - person[0], apple[1] - person[1]) < (required_distance - goal_error):
#            #if this is the case, an apple is within range of a person, including some consideration of error
#            apples_done.append(apple[0])
#            people_done.append(person[0])
#            pairs.append((person, apple))
#            
#people_not_done = [n for n in person_gt if n[0] not in people_done]
#apples_not_done = [n for n in apple_gt if n[0] not in apples_done]

#print(people_not_done, "\n", apples_not_done)

# assemble all combinations of people and apples who are not matched

#unmatched_pairs = []
#unmatched_person_ids = [n for n in person]

#for person in people_not_done:
#    for apple in apples_not_done:
#        unmatched_pairs.append((person, apple, np.hypot(apple[0] - person[0], apple[1] - person[1])))

#print(unmatched_pairs)

# pick combination of unmatched pairs which minimises travel distance


#list1 = ["a", "b", "c"]
#list2 = [1, 2]

#all_combinations = [list(zip(each_permutation, list2)) for each_permutation in itertools.permutations(list1, len(list2))]
#print(all_combinations)



## something new.

    
def generate_fruit_path(unpaired_apple_input, unpaired_person_input, bad_lemon_input, all_obstacles, initial_point, iterations):
    
    # initial conditions
    best_sol = ([], np.inf)
    
    iterations = 20
    for i in range(iterations):
        # try a path combination
        #unpaired_apple = [] # list of apples not yet paired
        #unpaired_person = [] # list of people without apple
        bad_lemon = bad_lemon.copy() # list of lemons requiring moving

        #done_apple = False
        done_lemon = False
        # build path that pairs people with apples, removes lemons, compute total distance

        seq = []
        seq.append(initial_point)

        while True:
            # move apple to person or move lemon away
            #flag = np.random.randint(0,2)
            seq_int = []
     #       print("here")
            #if flag and not done_apple:
            #    if not len(unpaired_person) or not len(unpaired_apple):
            #        done_apple = True
            #    else:
            #        # move apple to person
            #        random.shuffle(unpaired_person)
            #        unpaired_person.pop()
            #    
            #        random.shuffle(unpaired_apple)
            #        unpaired_apple.pop()

                    # random apple and person, move apple to person

            #        seq_int = move_apple_to_person()

            if not done_lemon:
                # move random lemon away

                if not len(bad_lemon):
                    done_lemon = True
                else:
                    random.shuffle(bad_lemon)
                    the_lemon = bad_lemon.pop()

                    potential_lemon_push_pos = random.choice(push_bad_lemon_away(the_lemon, all_obstacles)) # just take first one for now

     #               print(potential_lemon_push_pos)
                    ## RRT to lemon, push straight
    #                print("seq", np.array(seq[-1]))

                    rrt_path = None

                    rrt_res = RRT(
                        start=seq[-1],
                        goal=potential_lemon_push_pos, 
                        width=1.4, height=1.4, 
                        obstacle_list=all_obstacles, 
                        expand_dis=0.2, path_resolution=0.04,
                        max_points=50
                    )

                    while rrt_path is None:
                        rrt_path = rrt_res.planning()

    #                print("aaa")

                    rrt_path.reverse()

                    # just push it 0.5m for now
                    #print(to_lemon[-1], the_lemon)

                    dest = push_lemon_x(rrt_path[-1], the_lemon[0:2], 0.5)


                    seq_int = rrt_path + [dest]

            #print(done_apple, done_lemon)

            if done_lemon: #and done_lemon:
                break
            else:
                seq += seq_int
                #print(np.array(seq))

        # done a sequence, compute sequence length

        length = compute_seq_length(seq)
    #    print(length)

        if length < best_sol[1]:
            best_sol = (seq, length)

    return np.array(best_sol[0])




r_true_apple = 0.075
r_true_lemon = 0.06
r_true_person = 0.19
r_true_marker = 0.1

scale = 0.08

r_true_apple += scale
r_true_lemon += scale
r_true_person += scale
r_true_marker += scale
all_obstacles = []
for entry in apple_gt:
    all_obstacles.append(CircleT(entry[0], entry[1], r_true_apple, 0))

for entry in lemon_gt:
    all_obstacles.append(CircleT(entry[0], entry[1], r_true_lemon, 1))

for entry in person_gt:
    all_obstacles.append(CircleT(entry[0], entry[1], r_true_person, 2))

for entry in marker_gt:
    all_obstacles.append(CircleT(entry[0], entry[1], r_true_marker, 3))
        

A = np.array([2,0])
B = np.array([0,0])

print(push_bad_lemon_away(lemon_gt, all_obstacles))

#print(find_collinear_behind(B, A, 0.3))

#print(push_lemon_x(A, B, 0.5))

# run rrt pathing test
start = [-1, -1]
goal = [1, 1]

#rrt = RRT(start=start, goal=goal, width=1.4, height=1.4, obstacle_list=all_obstacles, expand_dis=0.3 , path_resolution=0.04, max_points=500)

#print(rrt.planning())
ani_path= animate_path_x(np.array([[0,0]]), (-1.5, 1.5), (-1.5, 1.5), all_obstacles)

lemon = 2


display(HTML("<style>.container { width:100% !important}</style>"))

[array([0.81215155, 0.77142775, 0.        ]), array([-0.72480112, -1.35942686,  1.        ]), array([ 0.13186204, -0.86565131,  2.        ])]
[ 0.55272734 -0.822936  ] [array([0.81215155, 0.77142775, 0.        ]), array([-0.72480112, -1.35942686,  1.        ])]


TypeError: 'bool' object is not iterable